# Merging East River Tree Stem Geolocation Points
**Author:** 'Marshall Worsham' <br>
**Creation Date:** '09/21/2020' <br>
**Revision Date:** '12/22/2020' <br>

---

## Contents

1 - [Front matter](#front)<br>
2 - [Libraries](#libraries)<br>
3 - [Import reference table](#import)<br>
4 - [Exploratory analysis](#eda)<br>
5 - [Rename and move](#rename)<br>
6 - [Prepare for append](#prep)<br>
7 - [Append](#append)<br>

---


## Front matter<a id='front'></a>

This notebook contains markdown and code for post-processing point shapefiles generated from Trimble Geo7X GPS acquisitions in the East River domain. The script appends the `Site` name and `subdirectory` to each shapefile name, then selects all projected point shapefiles, groups them by `Site` name, and merges points from the same site. The result is a set of shapefiles containing tree geolocation points, one set for each site in the watershed where stem geolocations were acquired from 2018–2020. Most output files contain some extraneous points marking corners and errata, which are cleaned out in '00_EastRiver_Clean_Tree_GPSPoints.ipynb'. 

The script was developed in `Python 3.8.2` on a Macbook Pro 2014 running OSX 10.14.6.


## Libraries<a id='libraries'></a>

In [ ]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import math
import re
from matplotlib import pyplot as plt
from os.path import join, getsize
%matplotlib inline
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

Define the working directory and list contents

In [ ]:
os.getcwd()
directory = os.sep.join(['/Volumes', 'GoogleDrive', 'My Drive', 'Research', 'RMBL', 'Working_Files', 'Forest_Inventory_Dataset'])
source_dir = os.sep.join([directory, 'Source'])
scratch_dir = os.sep.join([directory, 'Scratch'])
out_dir = os.sep.join([directory, 'Output'])
gps_dir = os.sep.join(['/Volumes', 'GoogleDrive', 'My Drive', 'Research', 'RMBL', 'RMBL-East River Watershed Forest Data', 'Data', 'Inventory Plots', '02_Inventory_Plots_GPS_Data', '2021'])
os.listdir(gps_dir)[0:10]

## Import reference table<a id='reference'></a>
First we import a CSV describing filenames and associated sites. Then we slice to create a simple list of filenames and the site at which the data inside those files were acquired.


In [ ]:
gps_index = pd.read_csv(os.sep.join([source_dir, 'EastRiver_GPS_Data_Index.csv']))
gps_index.dropna(0, subset=['Site', 'Contents'], inplace=True)
gps_index.loc[:, 'Filename':'Site'].tail(10)

## Exploratory analysis<a id='eda'></a>
Some simple exploratory analysis reveals see how many unique files are associated with each site.

In [ ]:
gps_index.groupby('Site').count()['Filename']

In [ ]:
# Scratch to set up the syntax for the function below that will relate filenames in the directory to filenames and site associations in the index dataframe
gps_index_sites = gps_index.loc[:,'Filename':'Site']
gps_index_sites.loc[gps_index_sites['Filename'] == 'WORSHAMM071610A'].iloc[0, 1]

List the filenames in all subdirectories of `directory` by walking the subdirectories and string-splitting on the last `/` in the path to isolate filenames. 

In [ ]:
for subdir, dirs, files in os.walk(os.sep.join([gps_dir, '2021'])):
    for filename in files:
        subdir_name = subdir.rsplit('/', 1)[-1]
        print(subdir_name)

## Rename and move<a id='rename'></a>
The function below finds the name of the `subdirectory` that each shapefile lives in and finds the `Site` with which that subdirectory is associated. The function renames each shapefile by appending the `subdirectory` name and `Site` name to the filename, then moves all files into a single directory.

In [ ]:
gps_dir

In [ ]:
for subdir, dirs, files in os.walk(gps_dir):
    for filename in files:
        gps_index_sites = gps_index.loc[:, 'Filename':'Site']
        subdir_name = subdir.rsplit('/', 1)[-1]
        index_sitename = str(gps_index_sites.loc[gps_index_sites['Filename'] == subdir_name, 'Site'].values).strip("[]").strip("'")
        newname = subdir_name + '_' + index_sitename + '_' + filename
        oldpath = subdir + os.sep + filename
        newpath = subdir + os.sep + newname
        #print(oldpath)
        #print(newname)
        #print(newpath)
        os.rename(oldpath, newpath)
        if not re.search('Line.+', filename) and not re.search('Area.+', filename) and not re.search('Icon.+', filename):
            print(newpath)

## Prepare for append<a id='prep'></a>

In [ ]:
# list all files
renamed_dir = os.path.join(scratch_dir, 'GPS_Data_2021_Renamed')
allfiles = os.listdir(renamed_dir)
allfiles[0:10]
#allfiles

In [ ]:
# generate a list of unique site names represented in the dataset
sitelist = gps_index['Site'].unique().tolist()
sitelist

In [ ]:
# filter only shapefiles containing point data types in the correct projection
# the target filenames will contain the tag "Project" AND either of the strings "Student" or "Point"
# filenames without "Student" and filenames containing "Area" and "Line" strings will be filtered out
point_str = 'Point_'
stud_str = 'Student'
proj_str = '_Project'
sf_allpoint = [i for i in allfiles if ((point_str in i or stud_str in i) and proj_str in i)]
print(len(allfiles))
print(len(sf_allpoint))
sf_allpoint

In [ ]:
# # manipulate the gps_index dataframe
# notcorners = gps_index[~gps_index['Contents'].str.contains('corner')] # filter out names of subdirs containing corners
# notcorners = notcorners['Filename'].to_list()
# print(notcorners[:10])
# print(len(sf_allpoint))
# print(len(notcorners))

In [ ]:
# manipulate gps_index dataframe
trees = gps_index[gps_index['Contents'].str.contains('tree')]
trees = trees['Filename'].to_list()
print(trees)

In [ ]:
# find files representing trees only, with .shp extension
trees_allfiles = [i for i in sf_allpoint if any(ii in i for ii in trees)]
trees_sf = [t for t in trees_allfiles if t.endswith('.shp')]

In [ ]:
print(len(trees_sf))
print(trees_sf)

## Append<a id='append'></a>

1. group files according to site name by finding common value from sitelist in `matches` string
2. for each site, select the first shapefile and assign it as base object
3. append all other shapefiles to the base object with `gpd.append()`
4. project crs to wgs84 utm zone 13
4. export the gpdf as a shapefile named: sitelist[i] + '_' + 'TreeStem_pts_WGS84UTM13.shp'

In [ ]:
# add full path to all filenames
trees_sf_paths = [renamed_dir + os.sep + i for i in trees_sf]
trees_sf_paths[-5:]

In [ ]:
# group files by plot
trees_sf_grouped = [[s for s in trees_sf_paths if key in s] for key in set(sitelist)]
trees_sf_grouped = [i for i in trees_sf_grouped if len(i) != 0] # filter out a few artifact empty lists

In [ ]:
# output list of grouped tree shapefiles in directory
trees_sf_grouped

In [ ]:
# aggregate, import, and append
alltrees_gpdf = []
for thing in trees_sf_grouped:
    site_gpdf = []
    for i in thing:
        gpdf = gpd.read_file(i)
        site_gpdf.append(gpdf)
    alltrees = site_gpdf[0].append(site_gpdf[1:])
    alltrees.to_crs(epsg = 32613, inplace = True)
    alltrees = alltrees.loc[alltrees.geom_type == 'Point']
    site = [s for s in sitelist if s in thing[0]][0]
    alltrees['Site'] = site
    alltrees.to_file(os.path.join(scratch_dir, 'GPS_Data_2021_MERGEDBYPLOT', site + '.shp'))
    alltrees_gpdf.append(alltrees)

In [ ]:
alltrees_gpdf